In [1]:
#install library
import matplotlib.pyplot as plt
import glob
import cv2
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from PIL import Image
from tensorflow.keras import layers, models
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import OneHotEncoder 
import os

/Users/kaiyuyokoi/.pyenv/versions/3.7.5/lib/python3.7/site-packages/pandas/compat/__init__.py:124: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)
2022-10-16 14:26:45.458655: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
encoder = OneHotEncoder()
encoder.fit([[0], [1]]) 
paths = []
data = []
result = []

for r, d, f in os.walk('/Users/kaiyuyokoi/Desktop/braintumor/no'):
    for file in f:
        if '.jpg' in file:
            paths.append(os.path.join(r, file))

for path in paths:
    img = Image.open(path)
    img = img.resize((128,128))
    img = np.array(img)
    if(img.shape == (128,128,3)):
        data.append(np.array(img))
        result.append(encoder.transform([[1]]).toarray())
    

In [3]:
# This cell updates result list for images without tumor

paths = []
for r, d, f in os.walk('/Users/kaiyuyokoi/Desktop/braintumor/yes'):
    for file in f:
        if '.jpg' in file:
            paths.append(os.path.join(r, file))

for path in paths:
    img = Image.open(path)
    img = img.resize((128,128))
    img = np.array(img)
    if(img.shape == (128,128,3)):
        data.append(np.array(img))
        result.append(encoder.transform([[0]]).toarray())

In [4]:
data = np.array(data)
data.shape

(138, 128, 128, 3)

In [6]:
result = np.array(result)
result = result.reshape(139,2)

ValueError: cannot reshape array of size 276 into shape (139,2)

In [57]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense, BatchNormalization
from keras.utils import np_utils
import keras
import numpy as np
from tensorflow.keras import optimizers

In [58]:
def train(x_train, x_test, y_train, y_test):
    model = Sequential()

    model.add(Conv2D(32, kernel_size=(2, 2), input_shape=(128, 128, 3), padding = 'Same'))
    model.add(Conv2D(32, kernel_size=(2, 2),  activation ='relu', padding = 'Same'))


    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, kernel_size = (2,2), activation ='relu', padding = 'Same'))
    model.add(Conv2D(64, kernel_size = (2,2), activation ='relu', padding = 'Same'))

    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
    model.add(Dropout(0.25))

    model.add(Flatten())

    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(2, activation='softmax'))

    model.compile(loss = "categorical_crossentropy", optimizer='Adamax')
    model.fit(x_train, y_train, epochs = 30, batch_size = 40, verbose = 1, validation_data = (x_test, y_test))
    model.save('./cnn.h5')
    return model 

def main():
    x_train,x_test,y_train,y_test = train_test_split(data, result, test_size=0.2, shuffle=True, random_state=0)
    model = train(x_train, x_test, y_train, y_test)

main()

Epoch 1/30
3/3 [==============================] - 5s 1s/step - loss: 25.2199 - val_loss: 147.3713
Epoch 2/30
3/3 [==============================] - 3s 1s/step - loss: 26.7084 - val_loss: 13.8383
Epoch 3/30
3/3 [==============================] - 4s 1s/step - loss: 8.8102 - val_loss: 4.8174
Epoch 4/30
3/3 [==============================] - 4s 1s/step - loss: 5.6163 - val_loss: 21.8787
Epoch 5/30
3/3 [==============================] - 3s 1s/step - loss: 4.8778 - val_loss: 19.4675
Epoch 6/30
3/3 [==============================] - 3s 1s/step - loss: 2.2329 - val_loss: 4.7536
Epoch 7/30
3/3 [==============================] - 3s 880ms/step - loss: 1.4322 - val_loss: 1.4194
Epoch 8/30
3/3 [==============================] - 3s 836ms/step - loss: 0.8505 - val_loss: 1.4725
Epoch 9/30
3/3 [==============================] - 3s 1s/step - loss: 1.3102 - val_loss: 2.4435
Epoch 10/30
3/3 [==============================] - 3s 1s/step - loss: 0.2758 - val_loss: 4.4880
Epoch 11/30
3/3 [===================